# Crime "Prediction"

This is a striped down version of the code I used to prep my entry for the NIJ Crime Forecasting Challenge. I wrote about my process and thinking in [this article](https://lawyerist.com/?p=185613).

In [1]:
import copy
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import geopandas as gp
from geopandas.tools import sjoin

In [2]:
def score(pred,act):
    act = act.drop('hotspot', 1)
    pred = pred.drop('calls', 1)
    pred = pred.drop('area', 1)
    test = pd.merge(pred, act, on='ID')
    PAI = (test[test["hotspot"]==1]["calls"].sum()/test["calls"].sum())/(test[test["hotspot"]==1]["area"].sum()/test["area"].sum())
    PAI_p = (test['calls'].nlargest(len(test[test["hotspot"]==1])).sum()/test["calls"].sum())/(test[test["hotspot"]==1]["area"].sum()/test["area"].sum())
    PEI = test[test["hotspot"]==1]["calls"].sum()/test['calls'].nlargest(len(test[test["hotspot"]==1])).sum()
    return PAI,PEI

In [3]:
# read shapefile http://gis.stackexchange.com/a/145029 
points_1 = gp.GeoDataFrame.from_file('../data/030112_123112_Data/NIJ2012_MAR01_DEC31.shp')
points_2 = gp.GeoDataFrame.from_file('../data/010113_123113_Data/NIJ2013_JAN01_DEC31.shp')
points_3 = gp.GeoDataFrame.from_file('../data/010114_123114_Data/NIJ2014_JAN01_DEC31.shp')
points_4 = gp.GeoDataFrame.from_file('../data/010115_123115_Data/NIJ2015_JAN01_DEC31.shp')
points_5 = gp.GeoDataFrame.from_file('../data/010116_073116_Data/NIJ2016_JAN01_JUL31.shp')
points_6 = gp.GeoDataFrame.from_file('../data/080116_083116_Data/NIJ2016_AUG01_AUG31.shp')
points_7 = gp.GeoDataFrame.from_file('../data/090116_093016_Data/NIJ2016_SEP01_SEP30.shp')
points_8 = gp.GeoDataFrame.from_file('../data/100116_103116_Data/NIJ2016_OCT01_OCT31.shp')
points_9 = gp.GeoDataFrame.from_file('../data/110116_113016_Data/NIJ2016_NOV01_NOV30.shp')
points_10 = gp.GeoDataFrame.from_file('../data/120116_123116_Data/NIJ2016_DEC01_DEC31.shp')
points_11 = gp.GeoDataFrame.from_file('../data/010117_013117_Data/NIJ2017_JAN01_JAN31.shp')
points_12 = gp.GeoDataFrame.from_file('../data/020117_021417_Data/NIJ2017_FEB01_FEB14.shp')
points_13 = gp.GeoDataFrame.from_file('../data/021517_022117_Data/NIJ2017_FEB15_FEB21.shp')

# join all together
points = pd.concat([points_1,points_2,points_3,points_4,points_5,points_6,points_7,
                    points_8,points_9,points_10,points_11,points_12,points_13])

# reset index to get rid of duplicates
points.reset_index(inplace=True)

# load polygons
poly = gp.GeoDataFrame.from_file('../data/sample_submission/NAME/ACFS/1MO/NAME_ACFS_1MO.shp')
poly['ID'] = poly.index

In [4]:
print("Number of polygons: %s"%len(poly))
print("Total Area of polygons: %s sq miles"%(poly["area"].sum()* 0.000000035870))
poly[:5].sort_values(by='area',ascending=False)

Number of polygons: 67372
Total Area of polygons: 147.703442594 sq miles


,area,geometry,hotspot,ID
4,41396,"POLYGON ((7644004.588155021 651395.9719231302, 7643754.588155014 651403.97313086, 7643754.588155014 651565.5577585897, 7644004.588155021 651565.5577585897, 7644004.588155021 651395.9719231302))",0,4
1,23919,"POLYGON ((7638004.588155022 651467.6146588892, 7637754.588155015 651472.145575754, 7637754.588155015 651565.5577585897, 7638004.588155022 651565.5577585897, 7638004.588155022 651467.6146588892))",0,1
3,9016,"POLYGON ((7643754.588155014 651403.97313086, 7643696.425960201 651405.8346020825, 7643700.533751362 651565.5577585897, 7643754.588155014 651565.5577585897, 7643754.588155014 651403.97313086))",0,3
2,8247,"POLYGON ((7638004.588155022 651467.6146588892, 7638004.588155022 651565.5577585897, 7638090.238925409 651565.5577585897, 7638086.059120219 651466.1381062157, 7638004.588155022 651467.6146588892))",0,2
0,184,"POLYGON ((7637754.588155015 651472.145575754, 7637751.356830861 651472.204139201, 7637753.873100898 651565.5577585897, 7637754.588155015 651565.5577585897, 7637754.588155015 651472.145575754))",0,0


In [5]:
print("Number of points: %s"%len(points))
points[:5]

Number of points: 954586


,index,CALL_GROUP,CASE_DESC,CATEGORY,census_tra,final_case,geometry,occ_date,x_coordina,y_coordina
0,0,DISORDER,DISTURBANCE - PRIORITY,STREET CRIMES,4900.0,DISTP,POINT (7641076 684831),2012-03-01,7641076.0,684831.0
1,1,DISORDER,DISTURBANCE - PRIORITY,STREET CRIMES,10600.0,DISTP,POINT (7642640 683167),2012-03-01,7642640.0,683167.0
2,2,DISORDER,DISTURBANCE - PRIORITY,STREET CRIMES,10600.0,DISTP,POINT (7643599 683216),2012-03-01,7643599.0,683216.0
3,3,DISORDER,DISTURBANCE - PRIORITY,STREET CRIMES,3502.0,DISTP,POINT (7644359 693642),2012-03-01,7644359.0,693642.0
4,4,DISORDER,DISTURBANCE - PRIORITY,STREET CRIMES,10600.0,DISTP,POINT (7644771 683859),2012-03-01,7644771.0,683859.0


In [6]:
def build(df_1,df_2):
    poly = copy.deepcopy(df_1)
    df = copy.deepcopy(df_2)
    pointInPolys = sjoin(poly,df,how='left', op='intersects')
    pointSumByPoly = pointInPolys.groupby('ID').size()
    output = pd.DataFrame(pointSumByPoly, columns=['calls'])
    output.index.name=None
    output['ID']=output.index
    output = poly.merge(output)
    output[['calls']] = output[['calls']].apply(pd.to_numeric)    
    return output

In [7]:
def maximize(cat,maxwhat):
    print("")
    print(cat)
    if cat == 'ALL':
        train_df = build(poly,points_train[points_train['CATEGORY']!='OTHER'])
    else:
        train_df = build(poly,points_train[points_train['CATEGORY']==cat])

    final_output = copy.deepcopy(train_df)
    final_output['hotspot'] = 1
    if maxwhat == 'PAI':
        final_output.ix[~final_output.index.isin(final_output["calls"].nlargest(113).index),'hotspot'] = 0
    else:
        final_output.ix[~final_output.index.isin(final_output["calls"].nlargest(334).index),'hotspot'] = 0
    print("Predicted area: %s sq mi (must be between 0.25 sq mi and 0.75 sq mi)"%(float(final_output[final_output['hotspot']==1]['area'].sum())* 0.000000035870))
    print("Number of polyons: %s"%len(final_output))
    print ("Total Area of polygons: %s sq miles"%(final_output["area"].sum()* 0.000000035870))
        
    return final_output

In [8]:
points_train = points[((points['occ_date']>= '2016-02-21') & (points['occ_date']<= '2017-02-21'))]

print("Train size: %s points"%len(points_train))

Burg = maximize('BURGLARY','PEI')
df = gp.GeoDataFrame(Burg, geometry='geometry')
df.to_file('ANDERTON/Burg/1WK/ANDERTON_Burg_1WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/Burg/2WK/ANDERTON_Burg_2WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/Burg/1MO/ANDERTON_Burg_1MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/Burg/2MO/ANDERTON_Burg_2MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/Burg/3MO/ANDERTON_Burg_3MO.shp', driver='ESRI Shapefile')

ACFS = maximize('ALL','PEI')
df = gp.GeoDataFrame(ACFS, geometry='geometry')
df.to_file('ANDERTON/ACFS/1WK/ANDERTON_ACFS_1WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/ACFS/2WK/ANDERTON_ACFS_2WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/ACFS/1MO/ANDERTON_ACFS_1MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/ACFS/2MO/ANDERTON_ACFS_2MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/ACFS/3MO/ANDERTON_ACFS_3MO.shp', driver='ESRI Shapefile')

SC = maximize('STREET CRIMES','PEI')
df = gp.GeoDataFrame(SC, geometry='geometry')
df.to_file('ANDERTON/SC/1WK/ANDERTON_SC_1WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/SC/2WK/ANDERTON_SC_2WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/SC/1MO/ANDERTON_SC_1MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/SC/2MO/ANDERTON_SC_2MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/SC/3MO/ANDERTON_SC_3MO.shp', driver='ESRI Shapefile')

TOA = maximize('MOTOR VEHICLE THEFT','PEI')
df = gp.GeoDataFrame(TOA, geometry='geometry')
df.to_file('ANDERTON/TOA/1WK/ANDERTON_TOA_1WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/TOA/2WK/ANDERTON_TOA_2WK.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/TOA/1MO/ANDERTON_TOA_1MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/TOA/2MO/ANDERTON_TOA_2MO.shp', driver='ESRI Shapefile')
df.to_file('ANDERTON/TOA/3MO/ANDERTON_TOA_3MO.shp', driver='ESRI Shapefile')

df[:5]

Train size: 209723 points

BURGLARY
Predicted area: 0.60260125743 sq mi (must be between 0.25 sq mi and 0.75 sq mi)
Number of polyons: 67372
Total Area of polygons: 147.703442594 sq miles

ALL
Predicted area: 0.7469880510300001 sq mi (must be between 0.25 sq mi and 0.75 sq mi)
Number of polyons: 67372
Total Area of polygons: 147.703442594 sq miles

STREET CRIMES
Predicted area: 0.7469880510300001 sq mi (must be between 0.25 sq mi and 0.75 sq mi)
Number of polyons: 67372
Total Area of polygons: 147.703442594 sq miles

MOTOR VEHICLE THEFT
Predicted area: 0.6769047787200001 sq mi (must be between 0.25 sq mi and 0.75 sq mi)
Number of polyons: 67372
Total Area of polygons: 147.703442594 sq miles


,area,geometry,hotspot,ID,calls
0,184,"POLYGON ((7637754.588155015 651472.145575754, 7637751.356830861 651472.204139201, 7637753.873100898 651565.5577585897, 7637754.588155015 651565.5577585897, 7637754.588155015 651472.145575754))",1,0,1
1,23919,"POLYGON ((7638004.588155022 651467.6146588892, 7637754.588155015 651472.145575754, 7637754.588155015 651565.5577585897, 7638004.588155022 651565.5577585897, 7638004.588155022 651467.6146588892))",1,1,1
2,8247,"POLYGON ((7638004.588155022 651467.6146588892, 7638004.588155022 651565.5577585897, 7638090.238925409 651565.5577585897, 7638086.059120219 651466.1381062157, 7638004.588155022 651467.6146588892))",1,2,1
3,9016,"POLYGON ((7643754.588155014 651403.97313086, 7643696.425960201 651405.8346020825, 7643700.533751362 651565.5577585897, 7643754.588155014 651565.5577585897, 7643754.588155014 651403.97313086))",1,3,1
4,41396,"POLYGON ((7644004.588155021 651395.9719231302, 7643754.588155014 651403.97313086, 7643754.588155014 651565.5577585897, 7644004.588155021 651565.5577585897, 7644004.588155021 651395.9719231302))",1,4,1
